In [1]:
import json
import requests
from config import fed_api_key
from pprint import pprint
import pandas as pd
import numpy as np

In [2]:
commodities_dict= {'lumber':'WPU081','cement':'PCU32733273','steel':'WPU10170674','stone':'WPS1321','brick':'PCU32733132733106'}

In [3]:
#'https://api.stlouisfed.org/fred/series/observations?series_id=WPU0811&api_key=0a60bfbafed5ab7080d559305d53573a&file_type=json'
base_url='https://api.stlouisfed.org/fred/series/observations?series_id='
mid_url = '&api_key='
end_url='&file_type=json'
urls =[]
dummies = []

In [4]:
commodity_name=[]
dates = []
values = []

for key,value in commodities_dict.items():
    url = base_url + value + mid_url + fed_api_key + end_url
    response = requests.get(url)
    data_json = response.json()
    for data in data_json['observations']:
         dates.append(data['date'])
         values.append(data['value'])
         commodity_name.append(key)
df = pd.DataFrame({
     "Commodity": commodity_name,
     "date": dates,
     "values": values
})
df.to_csv('Resources/ConstructionCommodities.csv')

In [5]:
file = "Resources/ZilloHomeValueIndexZip.csv"
zhvi_df = pd.read_csv(file)
zhvi_df = zhvi_df[zhvi_df['State']=='TX']
zhvi_df['Country'] = 'US'
zhvi_df

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,Country
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,225325.0,...,354939.0,358995.0,363611.0,369082.0,375735.0,382697.0,391365.0,402167.0,413199.0,US
5,91940,5,77449,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,107110.0,...,206612.0,208843.0,211725.0,213130.0,215301.0,217929.0,223432.0,229436.0,235859.0,US
7,91733,7,77084,Zip,TX,TX,Houston,Houston-The Woodlands-Sugar Land,Harris County,106353.0,...,204784.0,206993.0,209809.0,211363.0,213561.0,215995.0,220991.0,226551.0,232848.0,US
8,93144,8,79936,Zip,TX,TX,El Paso,El Paso,El Paso County,90628.0,...,147287.0,148997.0,150717.0,152543.0,154337.0,156983.0,159868.0,162910.0,165737.0,US
12,92593,12,78660,Zip,TX,TX,Pflugerville,Austin-Round Rock,Travis County,164957.0,...,313724.0,319220.0,328431.0,341024.0,359850.0,378330.0,400738.0,419873.0,435000.0,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30396,91419,34050,76467,Zip,TX,TX,Paluxy,Dallas-Fort Worth-Arlington,Hood County,NaN,...,221983.0,222022.0,226732.0,234899.0,237255.0,231062.0,222216.0,213098.0,208768.0,US
30437,92905,34207,79326,Zip,TX,TX,Fieldton,NaN,Lamb County,NaN,...,NaN,147635.0,148631.0,148374.0,147707.0,144321.0,140736.0,139101.0,135622.0,US
30449,91921,34272,77428,Zip,TX,TX,Palacios,Bay City,Matagorda County,NaN,...,124313.0,124893.0,125597.0,125732.0,125820.0,125440.0,124752.0,125707.0,125869.0,US
30484,59147,34430,3575,Zip,TX,TX,Clifton,NaN,Bosque County,NaN,...,515026.0,521246.0,525828.0,533536.0,542749.0,550674.0,554168.0,558548.0,569863.0,US


In [6]:
zhvi_df = zhvi_df.drop(columns=['SizeRank','RegionID','RegionType','StateName'])
zhvi2_df = pd.melt(zhvi_df,id_vars=['Country','RegionName','Metro','City','State','CountyName'],var_name='MeasureDate', value_name='MedianPrice')
zhvi2_df = zhvi2_df.rename(columns={"State": "UsState"})
zhvi2_df = zhvi2_df.rename(columns={"RegionName": "Zipcode"})
zhvi2_df

,Country,Zipcode,Metro,City,UsState,CountyName,MeasureDate,MedianPrice
0,US,77494,Houston-The Woodlands-Sugar Land,Katy,TX,Harris County,2000-01-31,225325.0
1,US,77449,Houston-The Woodlands-Sugar Land,Katy,TX,Harris County,2000-01-31,107110.0
2,US,77084,Houston-The Woodlands-Sugar Land,Houston,TX,Harris County,2000-01-31,106353.0
3,US,79936,El Paso,El Paso,TX,El Paso County,2000-01-31,90628.0
4,US,78660,Austin-Round Rock,Pflugerville,TX,Travis County,2000-01-31,164957.0
...,...,...,...,...,...,...,...,...
466175,US,76467,Dallas-Fort Worth-Arlington,Paluxy,TX,Hood County,2021-08-31,208768.0
466176,US,79326,NaN,Fieldton,TX,Lamb County,2021-08-31,135622.0
466177,US,77428,Bay City,Palacios,TX,Matagorda County,2021-08-31,125869.0
466178,US,3575,NaN,Clifton,TX,Bosque County,2021-08-31,569863.0


In [7]:
zhvi2_df.to_csv('Resources/zhvi_zip.csv')